# 创建自己的图结构

In [101]:
from typing import List, Mapping, Any, Set


class Graph:
    def __init__(self):
        # key 是该节点的唯一标识。 key 等于待转换的图结构中节点的表示方式, 比如其他图中节点的表示方式是 1,2,3, 或者 'A', 'B', 'C', 那么 key 就是对应的值
        # value 是具体的节点。 value 就是我们利用待转换的图结构中的节点, 所创建出来的属于我们自己结构的节点。
        self.nodes: Mapping[Any, Node] = {}
        self.edges: Set[Edge] = set()


class Node:
    def __init__(self, val):
        self.val = val
        self.ind = 0
        self.outd = 0
        self.nexts: List[Node] = []
        self.edges: List[Edge] = []


class Edge:
    def __init__(self, fr: Node, to: Node, weight=0):
        self.fr = fr
        self.to = to
        self.weight = weight


In [102]:
""" 支持的结构:
matrix = [
    [权值, fr节点, to节点], # 单向边: 
]
"""
def generate_graph2(matrix: List[List[int]]):
    graph = Graph()
    for item in matrix: # 每一个 item 都是一条边
        weight, fr, to = item # 
        # 判断边的两个节点是否已存在, 不存在则新增
        if fr not in graph.nodes:
            graph.nodes[fr] = Node(fr)
        if to not in graph.nodes:
            graph.nodes[to] = Node(to)
        # 不管节点存不存在, 新的边肯定是要添加的, 节点的属性也是要更新的
        fr_node = graph.nodes.get(fr)
        to_node = graph.nodes.get(to)
        new_edge = Edge(fr_node, to_node, weight)
        graph.edges.add(new_edge) # 新的边
        fr_node.edges.append(new_edge) # fr 增加一个邻接边
        fr_node.nexts.append(to_node) # fr 增加一个邻接点
        fr_node.outd += 1 # fr 的出度 + 1
        to_node.ind += 1  # to 的入度 + 1
    return graph

In [103]:
""" 支持的结构:
matrix = [
    [节点, 节点], # 双向边
]
"""
def generate_graph(matrix):
    graph = Graph()
    for one_edge in matrix:
        n1, n2 = one_edge # 双向边
        # 添加新节点
        if n1 not in graph.nodes:
            graph.nodes[n1] = Node(n1)
        if n2 not in graph.nodes:
            graph.nodes[n2] = Node(n2)
        n1_node, n2_node = graph.nodes.get(n1), graph.nodes.get(n2)
        e1, e2 = Edge(n1_node, n2_node), Edge(n2_node, n1_node)
        # 添加边
        graph.edges.add(e1)
        graph.edges.add(e2)
        n1_node.edges.append(e1)
        n2_node.edges.append(e2)
        # 更新节点属性
        n1_node.nexts.append(n2_node)
        n2_node.nexts.append(n1_node)
        n1_node.outd += 1
        n1_node.ind += 1
        n2_node.outd += 1
        n2_node.ind += 1
    return graph

# TEST - 广度优先遍历 BFS

In [104]:
from queue import Queue


# 广度有限遍历, 先处理好当前节点的所有邻接点, 再去处理邻接点的邻接点。 已经处理过的节点不要再处理。
# 利用 queue 保存待处理的节点, 利用 set 保存已经处理过的节点
def bfs(node):
    if node is None:
        return
    queue = Queue()
    selected_node = set()
    queue.put(node)
    selected_node.add(node)
    while not queue.empty():
        n = queue.get()
        print(n.val, end=' ')  # 出队列时处理
        # 处理完后, 先处理该节点的所有邻接点, 所以将他们先全部添加到队列中
        for wait_node in n.nexts:
            # 处理过的节点不要再处理
            if wait_node not in selected_node:
                selected_node.add(wait_node)
                queue.put(wait_node)
    print('')


In [105]:
matrix = [
    ['A', 'B'],
    ['A', 'C'],
    ['A', 'D'],
    ['B', 'C'],
    ['D', 'C'],
    ['E', 'C'],
    ['D', 'F'],
]
graph = generate_graph(matrix)
bfs(graph.nodes.get('E'))

E C A B D F 


# TEST - 深度优先遍历 DFS

In [106]:
# 深度优先, 一直逮着一个往下处理, 没得处理了才回退(出栈)
def dfs(node):
    stack = []
    selected_node = set()
    stack.append(node)
    print(node.val, end=' ')  # 处理时刻
    selected_node.add(node)  # 处理后标记起来

    while 0 != len(stack):  # stack 为空说明都被处理过了
        n = stack.pop()  # 将处理过的拿走。
        # 逮着 n 继续往下处理
        for next_node in n.nexts:
            if next_node not in selected_node:  # 逮着一个没处理过的
                stack.append(n)  # 此时还无法确定 n 的 nexts 都处理过了, 所以重新将 n 压栈
                stack.append(next_node)  # 拿到 stack 等待处理
                print(next_node.val, end=' ')  # 处理时刻
                selected_node.add(next_node)  # 处理后标记
                break  # 逮着一个就马上处理, 这才是 "深度优先"
    print('')

In [107]:
matrix = [
    ['A', 'B'],
    ['A', 'C'],
    ['B', 'D'],
    ['B', 'E'],
]
graph = generate_graph(matrix)
dfs(graph.nodes.get('A'))

A B D E C 


# [拓扑排序](https://www.nowcoder.com/practice/88f7e156ca7d43a1a535f619cd3f495c)

In [108]:
import sys


def get_input():
    node_num, _ = sys.stdin.readline().split()
    node_num = int(node_num)

    # 下标就是节点, 值就是该节点的入度。
    nodes_ind = [0] * node_num
    # 下标就是节点, 值是该节点的邻接点集合。
    nodes_nexts = [[] for _ in range(node_num)]

    for line in sys.stdin:
        fr, to = line.split()
        fr, to = int(fr)-1, int(to)-1  # 输入的节点是从 1 开始
        nodes_ind[to] += 1
        nodes_nexts[fr].append(to)

    return nodes_ind, nodes_nexts


def topology(nodes_ind, nodes_nexts):

    zero_nodes = []  # 存储入度为 0 的节点
    result = []  # 存储拓扑排序后的节点

    # 初始化 zero_nodes
    for node, ind in enumerate(nodes_ind):
        if ind == 0:
            zero_nodes.append(node)

    if len(zero_nodes) == 0:
        return []

    while len(zero_nodes) != 0:
        node = zero_nodes.pop()
        result.append(node)
        for next_node in nodes_nexts[node]:
            nodes_ind[next_node] -= 1   # 消除该节点的入度影响
            if nodes_ind[next_node] == 0:   # 每次消除都应该会有新的节点入度为 0
                zero_nodes.append(next_node)

    return result if len(result) == len(nodes_ind) else []


def main():# if __name__ == "__main__":

    nodes_ind, nodes_nexts = get_input()
    result = topology(nodes_ind, nodes_nexts)

    print(
        ' '.join(str(node+1) for node in result)
        if 0 != len(result) else 
        '-1'
    )


# TEST - 拓扑排序

In [109]:
class Graph:
    def __init__(self):
        self.nodes = {}
class Node:
    def __init__(self, val):
        self.val = val
        self.ind = 0 # 入度
        self.nexts = [] # 邻接点 集合

In [110]:
def generate_graph(matrix):
    graph = Graph()
    for edge in matrix:
        fr, to = edge
        if fr not in graph.nodes:
            graph.nodes[fr] = Node(fr)
        if to not in graph.nodes:
            graph.nodes[to] = Node(to)
        fr_node, to_node = graph.nodes.get(fr), graph.nodes.get(to)
        fr_node.nexts.append(to_node)
        to_node.ind += 1
    return graph

In [111]:
def topology(graph):
    ind_map = {} # 存储各节点的入度
    zero_nodes = [] # 存储入度为 0 的节点
    result = [] # 存储排序后的节点值
    # 初始化 ind_map 和 zero_nodes
    for node in graph.nodes.values():
        ind_map[node] = node.ind
        if node.ind == 0:
            zero_nodes.append(node)
    if len(zero_nodes) == 0:
        return []
    while len(zero_nodes) != 0:
        node = zero_nodes.pop()
        result.append(node)
        for next_node in node.nexts:
            ind_map[next_node] -= 1  # 更新邻接点的入度
            if ind_map[next_node] == 0: 
                zero_nodes.append(next_node) # 更新 zero_nodes
    
    return result if len(result) == len(ind_map) else []

In [113]:
matrix = [
    ['F', 'E'],
    ['F', 'D'],
    ['E', 'B'],
    ['D', 'B'],
    ['C', 'A'],
    ['C', 'B'],
    ['B', 'A'],
]
graph =generate_graph(matrix)
for node in topology(graph):
    print(node.val, end=' ')

C F D E B A 

# TEST - 最小生成树 MST

In [137]:
class Graph:
    def __init__(self):
        self.nodes = {}
        self.edges = set()
class Node:
    def __init__(self, val):
        self.val = val 
class Edge:
    def __init__(self, fr, to, w):
        self.fr = fr 
        self.to = to 
        self.w = w # 权值
    def __lt__(self, other):
        return self.w < other.w

In [138]:
# [n1, n2, w] 双向边
def generate_graph(matrix):
    graph = Graph()
    for edge in matrix:
        n1, n2, w = edge 
        if n1 not in graph.nodes:
            graph.nodes[n1] = Node(n1)
        if n2 not in graph.nodes:
            graph.nodes[n2] = Node(n2)
        # 获取节点
        n1_node, n2_node = graph.nodes.get(n1), graph.nodes.get(n2)
        # 创建边, 两条!
        e1, e2 = Edge(n1_node, n2_node, w), Edge(n2_node, n1_node, w)
        # 添加边, 两个节点都要!
        graph.edges.add(e1); graph.edges.add(e2)
    return graph


## Kruskal

In [139]:
class SimpleUnionFind:
    def __init__(self, nodes):
        # key 是某个集合中的一个代表性节点, 随意
        # value 是节点集合, 表示这些节点都是同一个集合
        self.set_map = {}
        # 为每一个节点都创建一个集合, 此时每个节点都不连通
        for node in nodes:
            _set = set()
            _set.add(node)
            self.set_map[node] = _set

    def is_same_set(self, fr, to):
        fr_set = self.set_map.get(fr)
        to_set = self.set_map.get(to)
        return fr_set == to_set

    def union(self, fr, to):
        fr_set = self.set_map.get(fr)
        to_set = self.set_map.get(to)
        for to_node in to_set:
            fr_set.add(to_node)
            # 以后访问 to_set 中的每一个节点 to_node 时, 都是发现 to_node 和 fr_set 中的节点相同, 即属于同一个集合
            self.set_map[to_node] = fr_set


In [140]:
from queue import PriorityQueue

def kruskal(graph):
    # 为图中每一个节点都生成属于自己的集合
    union_find = SimpleUnionFind(graph.nodes.values())
    # 优先级队列, 按权值从小到大顺序存储图的边
    priority_queue = PriorityQueue()
    for edge in graph.edges:
        priority_queue.put((edge))
    mst = set() # 生成树
    while not priority_queue.empty():
        # 每次取出最小边
        edge = priority_queue.get()
        # 判断该边是否会导致生成树成环
        if not union_find.is_same_set(edge.fr, edge.to):
            # 不成环, 可以添加该边。
            mst.add(edge)
            # 加入该边后, 要将两段节点合并为一个集合
            union_find.union(edge.fr, edge.to)
    return mst 

In [145]:
matrix = [
    ['B', 'C', 3],
    ['A', 'B', 1],
    ['A', 'C', 2],
]
graph = generate_graph(matrix)
mst_edges = kruskal(graph)
for edge in mst_edges:
    print(edge.w)

1
2


# TEST - Dijkstra 算法 - 最短路径